# 3066

Sort elements to efficiently get the 2 mins, use heap to sort them each time an element is added

In [ ]:
class Solution:
    def minOperations(self, nums: List[int], k: int) -> int:
        heapq.heapify(nums)
        res = 0
        while len(nums) >= 2 and nums[0] < k:
            min1 = heapq.heappop(nums)
            min2 = heapq.heappop(nums)
            heapq.heappush(nums, min(min1, min2) * 2 + max(min1, min2))
            res+=1
        return res

# leet 1235

In [ ]:
class Solution:
    def jobScheduling(self, startTime: List[int], endTime: List[int], profit: List[int]) -> int:
        # Input: startTime = [1,2,3,4,6], endTime = [3,5,10,6,9], profit = [20,20,100,70,60]
        # 1,3, 20  - > 3, 1, 0+20
        # 2,5, 20
        # 3,10, 100 -> 10, 3, 120
        # 4,6, 70
        # 6,9, 60

        #___________brute-force_____________
        #T:2^n, M: n
        #bfs all options, bfs or dfs with index 
        # max_profit = 0
        # intervals = [(s, e, p) for s, e, p in zip(startTime, endTime, profit)]
        # intervals.sort()  # sort by start time

        # def backtrack(time, index, p):
        #     nonlocal max_profit
        #     for i in range(index, len(intervals)):
        #         if intervals[i][0] >= time:  # if the job starts at or after the current time
        #             max_profit = max(max_profit, p + intervals[i][2])
        #             backtrack(intervals[i][1], i + 1, p + intervals[i][2])

        #     return max_profit

        # return backtrack(0, 0, 0)

        #_______________optimize____________________
        # [(0,0),(0,0),(0,0),[0,0],0,0]
        # [[0,0], [3, 20],[5,20] [10, 120], [6, 90], [9, 150]]  iterate each time and take the first possible one
        # T: O(n**2), M: O(N)
        # intervals = [(s, e, p) for s, e, p in zip(startTime, endTime, profit)]
        # intervals.sort()  # sort by start time
        # best_overall = 0
        # dp = [[0,0] for _ in range(len(intervals)+1)]
        # for i in range(len(intervals)):
        #     s, e, p = intervals[i]
        #     best_p = p
        #     i = i+1
        #     for j in range(i-1, -1, -1):
        #         if s >= dp[j][0]:
        #             best_p = max(dp[j][1] + p, best_p)
        #             best_overall = max(best_p, best_overall)
        #             dp[i] = [e, best_p]  

        # return best_overall

        # Create a list of jobs and sort them by start time.
        jobs = sorted(zip(startTime, endTime, profit))
        
        # Initialize heap and max_profit.
        # Heap stores tuples: (end_time, profit)
        heap = []
        max_profit_so_far = 0
        
        for s, e, p in jobs:
            # Remove jobs from the heap that have ended by the current job's start time.
            # While there is at least one job that ended before s, update max_profit_so_far.
            while heap and heap[0][0] <= s:
                end_time, curr_profit = heapq.heappop(heap)
                max_profit_so_far = max(max_profit_so_far, curr_profit)
            
            # Add the current job: the accumulated profit if this job is taken.
            # Note: max_profit_so_far stores the best profit we could have before starting job s.
            heapq.heappush(heap, (e, max_profit_so_far + p))
        
        # After processing all jobs, some jobs might still be in the heap.
        # Their profit values are also potential answers.
        while heap:
            _, curr_profit = heapq.heappop(heap)
            max_profit_so_far = max(max_profit_so_far, curr_profit)
        
        return max_profit_so_far





        


# leet 621
secret: use heapq for task priority and a Deque for cooldown of tasks

In [ ]:
class Solution:
    def leastInterval(self, tasks: List[str], n: int) -> int:
        freqs = {}
        cooldown = Deque()
        pq = []
        time = 0

        for task in tasks:
            freqs[task] = freqs.get(task, 0) + 1

        for key, freq in freqs.items():
            heapq.heappush(pq, (-freq, key))

        while pq or cooldown:
            time += 1
            if pq:
                freq, letter = heapq.heappop(pq)

            #add to cooldown
            freq += 1
            if freq < 0:
                cooldown.append((time +n, (freq, key)))#first available time
         
            #process if possible
            if cooldown and cooldown[0][0] == time:
                _, el = cooldown.popleft()
                heapq.heappush(pq, el)

        return time

# 2163

In [ ]:
#brute force

# class Solution:
#     # O(n choose k * (k for path copy))
#     def combinations(self, nums, k):
#         res = []
#         def _combine(start, path):
#             if len(path) == k:
#                 res.append(path[:])
#                 return

#             if start == len(nums):
#                 return

#             #take
#             path.append(nums[start])
#             _combine(start+1, path)
#             path.pop()
#             #not take
#             _combine(start+1, path)
#         _combine(0, [])
#         return res

#     def minimumDifference(self, nums: List[int]) -> int:
#         n = len(nums) // 3
#         res = float("inf")
#         for combination in self.combinations(nums, n):
#             removed = set(combination)
#             valid = []
#             for el in nums:
#                 if el not in removed:
#                     valid.append(el)

#             first_side = valid[:n]
#             second_side = valid[n:]
#             diff = sum(first_side) - sum(second_side)
#             res = min(res, diff)
#         return res



class Solution:
    def minimumDifference(self, nums: List[int]) -> int:
        n3, n = len(nums), len(nums) // 3

        part1 = [0] * (n + 1)
        # max heap
        total = sum(nums[:n])
        ql = [-x for x in nums[:n]]
        heapq.heapify(ql)
        part1[0] = total

        for i in range(n, n * 2):
            total += nums[i]
            heapq.heappush(ql, -nums[i])
            total -= -heapq.heappop(ql)
            part1[i - (n - 1)] = total

        # min heap
        part2 = sum(nums[n * 2 :])
        qr = nums[n * 2 :]
        heapq.heapify(qr)
        ans = part1[n] - part2

        for i in range(n * 2 - 1, n - 1, -1):
            part2 += nums[i]
            heapq.heappush(qr, nums[i])
            part2 -= heapq.heappop(qr)
            ans = min(ans, part1[i - n] - part2)

        return ans